In [1]:
from emg_driver.emg_shield import EMGShieldController
from plotters.mpl import DynamicPlotter

In [35]:
controller.close()

In [2]:
controller = EMGShieldController()

In [98]:
plotter = DynamicPlotter(channels=2, x_range=5000,min_val=0,max_val=4000, linewidth=1.0)

In [37]:
def plot_data(controller, times):
    for i in range(0, times):
        plotter.plotdata([controller.read_data().channels[0]])

In [6]:
import numpy as np

In [10]:
def plot_data_filtered(controller, window_size, times):
    values = np.empty(window_size, int)
    for i in range(0, times):
        if i < window_size:
            values[i] = controller.read_data().get_channels()[0][0]
            continue
        ptp_val = np.ptp(values)
        plotter.plotdata([ptp_val])
        values[i % window_size] = controller.read_data().get_channels()[0][0]

In [4]:
def encode_data(controller, window_size, times):
    prev = 0
    first = 1
    last = 0
    zero = 0
    count = 0
    binary = []
    values = np.empty(window_size, int)
    for i in range(0, times):
        if i < window_size:
            values[i] = controller.read_data().get_channels()[0][0]
            continue
        ptp_val = np.ptp(values)
        if prev < 650 and ptp_val > 650:
            first = 1
            zero = 1
            count = 0
            binary.append(1)
            last = len(binary)
        elif zero and ptp_val < 250 and first != 0:
            count += 1
            if count >= 10*window_size:
                binary.append(0)
                count = 0
                zero = 0
        prev = ptp_val
        plotter.plotdata([ptp_val])
        values[i % window_size] = controller.read_data().get_channels()[0][0]
    return binary[0:last]

In [7]:
plotter = DynamicPlotter(channels=1, x_range=5000,min_val=0,max_val=2000, linewidth=1.0)
encode_data(controller, 50, 10000)

KeyboardInterrupt: 

In [8]:
controller.close()

AttributeError: 'EMGShieldController' object has no attribute 'close'